<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Set-cookies" data-toc-modified-id="Set-cookies-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Set cookies</a></span></li><li><span><a href="#Get-company-name-list" data-toc-modified-id="Get-company-name-list-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Get company name list</a></span></li><li><span><a href="#Get-firm-uid" data-toc-modified-id="Get-firm-uid-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Get firm uid</a></span></li><li><span><a href="#Get-basic-information" data-toc-modified-id="Get-basic-information-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Get basic information</a></span></li><li><span><a href="#Parse-basic-information" data-toc-modified-id="Parse-basic-information-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Parse basic information</a></span></li><li><span><a href="#Get-company-development-information" data-toc-modified-id="Get-company-development-information-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Get company development information</a></span></li><li><span><a href="#Parse-development-information" data-toc-modified-id="Parse-development-information-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Parse development information</a></span></li><li><span><a href="#Merge-two-dictionary-for-one-company" data-toc-modified-id="Merge-two-dictionary-for-one-company-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Merge two dictionary for one company</a></span></li><li><span><a href="#Combine-base-and-adv-info" data-toc-modified-id="Combine-base-and-adv-info-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Combine base and adv info</a></span></li><li><span><a href="#Write-into-excel-file" data-toc-modified-id="Write-into-excel-file-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Write into excel file</a></span></li></ul></div>

In [114]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [156]:
from bs4 import BeautifulSoup
from urllib import parse
import requests
from tqdm import tqdm_notebook
from time import sleep
import pandas as pd
import openpyxl
from typing import List
import pickle
from glob import glob
# from utils import *

## Set cookies

In [119]:
cookie = "QCCSESSID=r18mmd4r1lvs24cca9t43969b5; zg_did=%7B%22did%22%3A%20%2216bc94370d11b4-047d80b89838b1-37677e02-384000-16bc94370d2471%22%7D; UM_distinctid=16bc94371142a0-021542bc12af83-37677e02-384000-16bc9437115489; _uab_collina=156244976879017749455047; acw_tc=2ff6129815624497682152240ef19f61e0ffef38ea7b331d445687c36c; Hm_lvt_3456bee468c83cc63fb5147f119f1075=1562449769,1562911126; CNZZDATA1254842228=1120220450-1562445588-https%253A%252F%252Fwww.qichacha.com%252F%7C1563334157; hasShow=1; Hm_lpvt_3456bee468c83cc63fb5147f119f1075=1563338099; zg_de1d1a35bfa24ce29bbf2c7eb17e6c4f=%7B%22sid%22%3A%201563334592268%2C%22updated%22%3A%201563338100242%2C%22info%22%3A%201563154163097%2C%22superProperty%22%3A%20%22%7B%7D%22%2C%22platform%22%3A%20%22%7B%7D%22%2C%22utm%22%3A%20%22%7B%7D%22%2C%22referrerDomain%22%3A%20%22%22%2C%22cuid%22%3A%20%22af5c198de42103292f80181258ee27a7%22%7D"

## Get company name list

In [106]:
index_names_df = pd.read_csv('index_name.csv', \
                             sep='\t', \
                             header=None,  \
                             names=['id', 'name'])
company_list = index_names_df['name'].tolist()
print(len(company_list))

# we take 10 companies for testing
company_list = company_list[:10]

100


## Get firm uid

In [100]:
# define the header which includes cookies
header_uids = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36",
    "Accept": "text/html, */*; q=0.01",
    "Accept-Language": "en,zh-CN;q=0.9,zh;q=0.8,zh-HK;q=0.7",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "X-Requested-With": "XMLHttpRequest",
    ############# need to be modify ##########
    "Host": "www.qichacha.com",
    "Referer": "https://www.qichacha.com/",
    "Cookie": cookie
}

In [111]:
def get_firm_uid(header_uids: dict, name_list: list) -> List[str]:
    
    # define the request url for getting company uid
    uid_request_url_list = ["https://www.qichacha.com/search?key={}"\
                            .format(parse.quote(name)) for name in name_list]
    uid_list = []
    for url in tqdm_notebook(uid_request_url_list):
        response = requests.get(url, headers=header_uids)
        soup = BeautifulSoup(response.content)
        uid_list.append(soup.select_one("#searchlist table.m_srchList "\
                                        "tbody#search-result tr.frtrt "\
                                        "td.checktd label.text-dark-lter "\
                                        "input").get('value'))
    return uid_list

In [73]:
uid_list = get_firm_uid(header_uids, name_list=company_list)

In [107]:
uid_list[0]

'8276bd9bbae49966f0a0c2b708775df7'

## Get basic information

In [120]:
# define the header for getting basic information
header_basic_info = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36",
    "Accept": "text/html, */*; q=0.01",
    "Accept-Language": "en,zh-CN;q=0.9,zh;q=0.8,zh-HK;q=0.7",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "X-Requested-With": "XMLHttpRequest",
    ############# need to be modify ##########
    "Host": "www.qichacha.com",
    "Cookie": cookie
}

In [147]:
def get_basic_info_soup(header_basic_info: dict, \
                        uid_list: list) -> List[BeautifulSoup]:
    
    # generate basic information request url
    basic_info_request_url_list = ["https://www.qichacha.com/firm_{}.html"\
                                   .format(uid) for uid in uid_list]
    
    basic_soup_list = []
    for url in tqdm_notebook(basic_info_request_url_list):
        response = requests.get(url, headers=header_basic_info)
        basic_soup_list.append(BeautifulSoup(response.content))
    
    return basic_soup_list

In [122]:
basic_soup_list = get_basic_info_soup(header_basic_info, uid_list)

In [135]:
# save the basic_soup_list to files
for uid, basic_soup in tqdm_notebook(zip(uid_list, basic_soup_list), \
                                     total=len(basic_soup_list)):
    with open("basic_soup_list/{}.soup".format(uid), 'w') as f:
        f.write(str(basic_soup))
        
# read from files
basic_soup_list = []
for file_name in glob('basic_soup_list/*'):
with open(file_name, 'r') as f:
    basic_soup_list.append(BeautifulSoup(f.read()))

## Parse basic information

In [395]:
def parse_basic_info(basic_soup: BeautifulSoup) -> dict:
    
    info_dict = {}
    
    # a helper function 
    def _helper_fun_get_key_value_add_to_dict(_row, info_dict):
        _key = _row.find_all('td')[0].text.strip()
        _value = _row.find_all('td')[1].text.strip()
        info_dict[_key] = _value
        _key = _row.find_all('td')[2].text.strip()
        _value = _row.find_all('td')[3].text.strip()
        info_dict[_key] = _value
        return info_dict

    #
    # 工商信息
    #
    panel = basic_soup.select_one("#base_div #Cominfo")
    table1 = panel.select("table")[0]
    table2 = panel.select("table")[1]
    
    # 法定代表人信息
    info_dict["法定代表人信息"] = table1.find("h2", {'class': 'seo font-20'}).text
    
    # 注册资本, 实缴资本
    _row = table2.find_all('tr')[0]
    info_dict = _helper_fun_get_key_value_add_to_dict(_row, info_dict)
    
    # 经营状态, 成立日期
    _row = table2.find_all('tr')[1]
    info_dict = _helper_fun_get_key_value_add_to_dict(_row, info_dict)
    
    # 统一社会信用代码, 纳税人识别号
    _row = table2.find_all('tr')[2]
    info_dict = _helper_fun_get_key_value_add_to_dict(_row, info_dict)
    
    # 注册号, 组织机构代码
    _row = table2.find_all('tr')[3]
    info_dict = _helper_fun_get_key_value_add_to_dict(_row, info_dict)
    
    # 企业类型, 所属行业
    _row = table2.find_all('tr')[4]
    info_dict = _helper_fun_get_key_value_add_to_dict(_row, info_dict)
    
    # 核准日期, 登记机关
    _row = table2.find_all('tr')[5]
    info_dict = _helper_fun_get_key_value_add_to_dict(_row, info_dict)
    
    # 所属地区, 英文名
    _row = table2.find_all('tr')[6]
    info_dict = _helper_fun_get_key_value_add_to_dict(_row, info_dict)
    
    # 曾用名, 参保人数
    _row = table2.find_all('tr')[7]
    info_dict = _helper_fun_get_key_value_add_to_dict(_row, info_dict)
    
    # 人员规模, 营业期限
    _row = table2.find_all('tr')[8]
    info_dict = _helper_fun_get_key_value_add_to_dict(_row, info_dict)
    
    # 企业地址
    _row = table2.find_all('tr')[9]
    _key = _row.find_all('td')[0].text.strip()
    _value = _row.find_all('td')[1].text.strip().split('\n')[0]
    info_dict[_key] = _value
    
    # 经营范围
    _row = table2.find_all('tr')[10]
    _key = _row.find_all('td')[0].text.strip()
    _value = _row.find_all('td')[1].text.strip()
    
    #
    # 对外投资
    #
    panel = basic_soup.select_one("#base_div #touzilist")
    # 对外投资数量
    if panel == None:
        info_dict["对外投资数量"] = 'None'
    else:
        info_dict["对外投资数量"] \
        = panel.select_one("div").find("span", {'class': 'tbadge'}).text
    
    #
    # 分支机构
    #
    panel = basic_soup.select_one("#base_div #branchelist")
    # 分支机构数量
    if panel == None:
        info_dict["分支机构数量"] = 'None'
    else:
        info_dict["分支机构数量"] \
        = panel.select_one("div").find("span", {'class': 'tbadge'}).text
    
    return info_dict

## Get company development information

In [148]:
# define the header for getting development information
header_dev_info = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36",
    "Accept": "text/html, */*; q=0.01",
    "Accept-Language": "en,zh-CN;q=0.9,zh;q=0.8,zh-HK;q=0.7",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "X-Requested-With": "XMLHttpRequest",
    ############# need to be modify ##########
    "Host": "www.qichacha.com",
    "Cookie": cookie
}

In [149]:
def get_dev_info_soup(header_dev_info: dict, \
                      uid_list: list) -> List[BeautifulSoup]:

    # generate header for getting development information
    header_dev_info_list = [dict(header_dev_info, \
                             **{"Referer": "https://www.qichacha.com/firm_{}.html"\
                                .format(uid)}) for uid in uid_list]

    # # generate the request url for getting development information
    dev_request_url_list = ["https://www.qichacha.com/company_getinfos?"\
                            "unique={0}&"\
                            "companyname={1}&"\
                            "tab=report".format(uid, parse.quote(name)) \
                            for uid, name in zip(uid_list, company_list)]
    
    soup_dev_info_list = []
    for header, url in tqdm_notebook(zip(header_dev_info_list, dev_request_url_list), \
                                     total=len(header_dev_info_list)):
        response = requests.get(url, headers=header)
        soup_dev_info_list.append(BeautifulSoup(response.content))
    
    return soup_dev_info_list

In [150]:
soup_dev_info_list = get_dev_info_soup(header_dev_info, uid_list)

In [152]:
# save the soup_dev_info_list to files
for uid, dev_soup in tqdm_notebook(zip(uid_list, soup_dev_info_list), \
                                   total=len(soup_dev_info_list)):
    with open("dev_soup_list/{}.soup".format(uid), 'w') as f:
        f.write(str(dev_soup))
# read from files
soup_dev_info_list = []
for file_name in glob('dev_soup_list/*'):
    with open(file_name, 'r') as f:
        soup_dev_info_list.append(BeautifulSoup(f.read()))

## Parse development information

In [423]:
def parse_dev_info(dev_soup: BeautifulSoup) -> dict:
    
    info_dict = {}
    
    # a helper function 
    def _helper_fun_get_key_value_add_to_dict(_row, info_dict):
        _key = _row.find_all('td')[0].text.strip()
        _value = _row.find_all('td')[1].text.strip()
        info_dict[_key] = _value
        _key = _row.find_all('td')[2].text.strip()
        _value = _row.find_all('td')[3].text.strip()
        info_dict[_key] = _value
        return info_dict
    
    main_panel = soup_dev_info.find("div", {'id': 0})
    
    #
    # 企业基本信息
    #
    panel = main_panel.find_all('table')[0]
    
    # 注册号, 统一社会信用代码
    _row = panel.find_all('tr')[0]
    info_dict = _helper_fun_get_key_value_add_to_dict(_row, info_dict)
    
    # 企业经营状态, 企业联系电话
    _row = panel.find_all('tr')[1]
    info_dict = _helper_fun_get_key_value_add_to_dict(_row, info_dict)
    
    # 从业人数, 邮政编码
    _row = panel.find_all('tr')[2]
    info_dict = _helper_fun_get_key_value_add_to_dict(_row, info_dict)
    
    # 有限责任公司本年度是否发生股东股权转让, 企业是否有投资信息或购买其他公司股权
    _row = panel.find_all('tr')[3]
    info_dict = _helper_fun_get_key_value_add_to_dict(_row, info_dict)
    
    # 电子邮箱
    _row = panel.find_all('tr')[4]
    _key = _row.find_all('td')[0].text.strip()
    _value = _row.find_all('td')[1].text.strip()
    info_dict[_key] = _value
    
    # 企业通讯地址
    _row = panel.find_all('tr')[5]
    _key = _row.find_all('td')[0].text.strip()
    _value = _row.find_all('td')[1].text.strip().split('\n')[0]
    info_dict[_key] = _value
    
    #
    # 企业资产状况信息
    #
    panel = main_panel.find_all('table')[3]
    
    # 资产总额, 所有者权益合计
    _row = panel.find_all('tr')[0]
    info_dict = _helper_fun_get_key_value_add_to_dict(_row, info_dict)
    
    # 营业总收入, 利润总额
    _row = panel.find_all('tr')[1]
    info_dict = _helper_fun_get_key_value_add_to_dict(_row, info_dict)
    
    # 营业总收入中主营业务收入, 净利润
    _row = panel.find_all('tr')[2]
    info_dict = _helper_fun_get_key_value_add_to_dict(_row, info_dict)
    
    # 纳税总额, 负债总额
    _row = panel.find_all('tr')[3]
    info_dict = _helper_fun_get_key_value_add_to_dict(_row, info_dict)
    
    #
    # 社保信息
    #
    panel = main_panel.find_all('table')[4]
    
    # 城镇职工基本养老保险, 职工基本医疗保险
    _row = panel.find_all('tr')[0]
    info_dict = _helper_fun_get_key_value_add_to_dict(_row, info_dict)
    
    # 生育保险, 失业保险
    _row = panel.find_all('tr')[1]
    info_dict = _helper_fun_get_key_value_add_to_dict(_row, info_dict)
    
    # 工伤保险
    _row = panel.find_all('tr')[2]
    _key = _row.find_all('td')[0].text.strip()
    _value = _row.find_all('td')[1].text.strip().split('\n')[0]

    return info_dict

## Merge two dictionary for one company

In [431]:
info_dict = parse_basic_info(basic_soup)

In [432]:
info_dict.update(parse_dev_info(dev_soup))

## Combine base and adv info

In [ ]:
df = pd.concat([index_name_uid_df, \
                base_info_df.drop(columns=['name']), \
                adv_info_df], axis=1, sort=False)

In [ ]:
# add new columns: firm info site page
df['firm_info_site'] = df['uid'].map(lambda x: \
                                     "https://www.qichacha.com/firm_{}.html".format(x))
# save df to xlsx
df.to_excel('../all_df.xlsx')
# read from xlsx
# df = pd.read_excel('../all_df.xlsx', index_col=0)

## Write into excel file

In [ ]:
to_excel_file('../sample.xlsx', df)